In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('data/train.csv')

In [ ]:
train['Date'] = pd.to_datetime(train['Date'])
train['Week'] = train['Date'].dt.week

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
import numpy as np

def get_data(s, d):
    item = train[(train['Store']==s) & (train['Dept']==d)]
    X = item['Week'].values
    X = np.expand_dims(X, axis=1)
    Y = item['Weekly_Sales'].values
    return X.copy(), Y.copy()

In [ ]:
test = pd.read_csv('data/sampleSubmission.csv', sep='_')
test['Date'] = pd.to_datetime(test['Date'])
test['Week'] = test['Date'].dt.week

stores = test['Store'].unique()
depts_store = {}

for s in stores:
    depts_store[s] = test[test['Store']==s]['Dept'].unique()

In [ ]:
models = {}

for s in stores:
    for d in depts_store[s]:
        X, Y = get_data(s, d)
        if len(X) > 0:
            models[(s, d)] = KNeighborsRegressor(
                n_neighbors=len(X), 
                weights='distance',
                n_jobs=-1)
            models[(s, d)].fit(X, Y)
        else:
            models[(s, d)] = None

In [ ]:
from IPython.display import clear_output

test['Weekly_Sales'] = 0

for s in stores:
    for d in depts_store[s]:
        print(s, d)
        idx = test.index[(test['Store']==s) & (test['Dept']==d)].tolist()
        if models[(s, d)] != None:
            test.loc[idx, 'Weekly_Sales'] = models[(s, d)].predict(
                np.expand_dims(test.loc[idx, 'Week'], axis=1))
    clear_output()
test

In [ ]:
submit = pd.read_csv('data/sampleSubmission.csv')
submit['Weekly_Sales'] = test['Weekl_Sales']
submit.columns = ['Id', 'Weekly_Sales']
submit.to_csv('data/submit.csv', index=False)